In [1]:
import os
os.chdir("../")

In [2]:
%pwd

'/Users/sriram.ranganathan99/ccc/E2E-sentence-similarity-MLflow-DVX'

In [13]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class TrainingConfig:
    base_model: Path
    updated_model_path: Path
    params_epochs: int
    params_batch_size: int
    params_warmup_steps: int

In [3]:
from Sentence_similarity.utils.common import read_pickle, read_yaml
from Sentence_similarity.constants import CONFIG_FILE_PATH
model_path = read_yaml(CONFIG_FILE_PATH).base_model.local_model_file
model = read_pickle(model_path)

[2024-01-25 20:57:05,656: INFO: common: yaml file: config/config.yaml loaded successfully]


/Users/sriram.ranganathan99/ccc/cancer/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
import pandas as pd
from sentence_transformers import InputExample, losses
from torch.utils.data import DataLoader
import pickle


class SentenceSimilarityTrainer:
    def __init__(self, model, csv_file_path, batch_size=16):
        self.model = model
        self.csv_file_path = csv_file_path
        self.batch_size = batch_size

    def get_training_data(self):
        df = pd.read_csv(self.csv_file_path)
        examples = []
        for _, row in df.iterrows():
            examples.append(
                InputExample(
                    texts=[row["Sentence 1"], row["Sentence 2"]], label=float(row["Similarity"])
                )
            )
        return DataLoader(examples, shuffle=True, batch_size=self.batch_size)

    def train_model(self, epochs=10, warmup_steps=100):
        train_dataloader = self.get_training_data()
        train_loss = losses.CosineSimilarityLoss(self.model)
        self.model.fit(
            train_objectives=[(train_dataloader, train_loss)],
            epochs=epochs,
            warmup_steps=warmup_steps,
        )
        return model

In [11]:
trainer = SentenceSimilarityTrainer(model, read_yaml(CONFIG_FILE_PATH).data_ingestion.local_data_file)

[2024-01-25 21:01:11,511: INFO: common: yaml file: config/config.yaml loaded successfully]


In [12]:
trainer.train_model(epochs=5)

Epoch: 100%|██████████| 5/5 [00:02<00:00,  1.76it/s]


SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)